In [3]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt


In [4]:
daily_sales_numbers = [21,22,-108,31,-1,32,34,31]

In [5]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [7]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [8]:
for sales in tf_dataset.take(3):
    print(sales)

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)


In [9]:
tf_dataset = tf_dataset.filter(lambda x:x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [10]:
tf_dataset = tf_dataset.map(lambda x:x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [11]:
tf_dataset = tf_dataset.shuffle(3)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2232
2448
2304


In [12]:
"""datset divided into batches. Useful when you have multi gpu environment and run each batch in difference gpus"""
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1512 2304]
[1584 2232]
[2232 2448]


In [14]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset = tf_dataset.filter(lambda x:x>0).map(lambda y:y*72).shuffle(3).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1584 2304]
[2448 2232]
[1512 2232]


In [16]:
image_ds = tf.data.Dataset.list_files('images/*/*',shuffle=False)
for img in image_ds.take(5):
    print(img.numpy())


b'images\\cat\\Abyssinian_127_jpg.rf.b2553880d73a78e26e396fe7775f31be.jpg'
b'images\\cat\\Abyssinian_131_jpg.rf.3aeee0801499f335e3a59b51189e1351.jpg'
b'images\\cat\\Abyssinian_149_jpg.rf.b6b4a72a93f8bc2b239b18c5d181190c.jpg'
b'images\\cat\\Abyssinian_15_jpg.rf.7bc44eeb9865e35cb1d5f9ef4f53b381.jpg'
b'images\\cat\\Abyssinian_167_jpg.rf.1d367cb1cf82c1e5ebda79fda928cee5.jpg'


In [18]:
image_ds = image_ds.shuffle(200)
for img in image_ds.take(5):
    print(img.numpy())

b'images\\dog\\pomeranian_120_jpg.rf.89b41b134a347498f6f5593819f6214e.jpg'
b'images\\cat\\Persian_117_jpg.rf.30b82f0e3d2ae0115aaf62d09e792f1f.jpg'
b'images\\cat\\Egyptian_Mau_108_jpg.rf.afd725b697520d03a6d12b912e6333e6.jpg'
b'images\\cat\\Birman_144_jpg.rf.369cb41f763175f21c7cfd77e77509ec.jpg'
b'images\\dog\\keeshond_12_jpg.rf.81daa1618b12f7cdd987dc7887ed913a.jpg'


In [ ]:
class_names = ['cat','dog']

In [19]:
image_count = len(image_ds)
image_count

275

In [20]:
train_size = int(image_count*0.8)
train_ds = image_ds.take(train_size)
test_ds = image_ds.skip(train_size)

In [21]:
len(train_ds)

220

In [22]:
len(test_ds)

55

In [23]:
for img in image_ds.take(1):
    print(img.numpy())

b'images\\cat\\Sphynx_167_jpg.rf.caaec066ce0f7be7decfc147eef1e244.jpg'


In [30]:
# s = b'images\\cat\\Sphynx_167_jpg.rf.caaec066ce0f7be7decfc147eef1e244.jpg'
import os
def get_label(file_path):
    return tf.strings.split(file_path, os.path.sep)[-2]


In [33]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128,128])
    
    return img, label

In [32]:
for t in train_ds.take(4):
    print(t.numpy())

b'images\\dog\\havanese_12_jpg.rf.9a82100b6a15bb23792a4c6e53110636.jpg'
b'images\\dog\\pomeranian_183_jpg.rf.99ea2d18b1560ce9082579726ab211a9.jpg'
b'images\\dog\\samoyed_150_jpg.rf.7bc090c04cc30f98b1d7b597fbac029f.jpg'
b'images\\cat\\Birman_151_jpg.rf.179ae9238f29eda8d63af4da89c48e05.jpg'


In [37]:
train_ds = train_ds.map(process_image)
for img,label in train_ds.take(3):
    print("label: ", label)
    print("Image:", img)


label:  tf.Tensor(b'cat', shape=(), dtype=string)
Image: tf.Tensor(
[[[12.       16.       15.      ]
  [12.       16.       15.      ]
  [12.       16.       15.      ]
  ...
  [29.894531 40.89453  26.894531]
  [29.       40.       26.      ]
  [30.210938 41.210938 27.210938]]

 [[12.       16.       15.      ]
  [12.       16.       15.      ]
  [12.       16.       15.      ]
  ...
  [31.683594 42.683594 28.683594]
  [30.81955  41.81955  27.81955 ]
  [30.367188 41.367188 27.367188]]

 [[12.       16.       15.      ]
  [12.       16.       15.      ]
  [12.       16.       15.      ]
  ...
  [33.472656 44.472656 30.472656]
  [32.69925  43.69925  29.69925 ]
  [34.       45.       31.      ]]

 ...

 [[22.148438 22.148438 20.148438]
  [20.218842 20.218842 18.218842]
  [23.016876 23.016876 21.016876]
  ...
  [32.       40.       27.      ]
  [32.570312 40.570312 27.570312]
  [33.       41.       28.      ]]

 [[16.5047   19.789062 18.789062]
  [16.525818 19.746094 18.746094]
  [18.1445

In [38]:
def scale(image,label):
    return image/255, label

In [39]:
train_ds = train_ds.map(scale)
for image, label in train_ds.take(3):
    print("***image: ",image.numpy()[0][0])
    print("***label; ", label.numpy())

***image:  [0.0096201 0.0096201 0.0096201]
***label;  b'cat'
***image:  [0.00392157 0.00392157 0.        ]
***label;  b'dog'
***image:  [0.78152215 0.8599535  0.95407116]
***label;  b'dog'
